In [101]:
import pandas as pd

In [102]:
df = pd.read_excel("../data/TABLA_ACCIDENTES_22.xlsx", sheet_name=0) #el excel tiene dos hojas pero solo queremos la primera

df.head()

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,...,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA
0,1,2022,1,6,3,1,0,NaN,1,1,...,999,1,1,6,1,NaN,NaN,1,998,3
1,2,2022,1,5,16,1,1059,NaN,1,1,...,999,1,3,1,3,NaN,NaN,1,998,2
2,3,2022,1,6,21,1,0,NaN,1,1,...,0,1,1,6,2,NaN,NaN,1,998,2
3,4,2022,1,7,12,1,0,NaN,1,1,...,999,1,3,1,3,NaN,NaN,1,998,1
4,5,2022,1,7,12,1,0,NaN,1,1,...,999,1,1,1,1,NaN,NaN,1,998,3


In [103]:
df.shape

(97916, 73)

In [104]:
df.duplicated().sum()

0

Hay hasta 31 columnas indicando la gravedad del accidente, por lo que lo primero que haremos será agruparlas todas en una donde tan solo se indique la gravedad como leve, grave o mortal.

Lo primero para ello es entender nuestros datos, por ejemplo, ¿la columna que indica los fallecidos en 30 días integra la columna fallecidos en 24 horas o son independientes?

In [105]:
df.loc[df['TOTAL_MU24H'] > df['TOTAL_MU30DF']]

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,...,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA


In [106]:
#Hacemos lo mismo para el número total de víctimas
df.loc[df['TOTAL_VICTIMAS_24H'] > df['TOTAL_VICTIMAS_30DF']]

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,...,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA


No existe ninguna columna de las de 24 horas sea mayor que su respectiva columna a 30 días, se asume por tanto que las columnas en 30 días ya incluyen las de 24 horas.

No vamos a eliminar el resto de columnas sobre fallecidos ya que aunque no se vayan a incluir en nuestro modelo pueden ser interesantes para analizar durante el EDA.

Otra cuestión parecida a la anterior es si el número de heridos hospitalizados incluye también a los que acabaron muriendo. Este problema es más complicado, ya que si una víctima es hospitalizada y acaba mueriendo en los 30 días posteriores es posible que cuente dentro de los dos grupos, mientras que si una persona fallece en el instante del accidente no será hospitalizado y solo contará en el grupo de fallecidos

In [108]:
df[['TOTAL_HG24H','TOTAL_MU30DF','TOTAL_HG30DF']].loc[df['TOTAL_MU30DF'] > df['TOTAL_HG30DF']]

,TOTAL_HG24H,TOTAL_MU30DF,TOTAL_HG30DF
16,0,1,0
133,0,1,0
196,0,1,0
324,1,1,0
446,0,2,0
...,...,...,...
97610,0,1,0
97619,0,1,0
97622,0,1,0
97646,1,1,0


In [ ]:
#más hospitalizados en 24 horas que en 30 días
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF','TOTAL_VICTIMAS_24H',
       'TOTAL_VICTIMAS_30DF','TOTAL_HL24H','TOTAL_HL30DF']].loc[df['TOTAL_HG24H'] > df['TOTAL_HG30DF']]

,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF,TOTAL_VICTIMAS_24H,TOTAL_VICTIMAS_30DF,TOTAL_HL24H,TOTAL_HL30DF
324,0,1,1,0,1,1,0,0
605,0,1,1,0,1,1,0,0
620,0,1,4,3,4,4,0,0
1568,0,1,1,0,3,3,2,2
1763,0,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...
97226,0,1,1,0,1,1,0,0
97229,0,1,1,0,1,1,0,0
97506,0,1,1,0,1,1,0,0
97646,0,1,1,0,1,1,0,0


In [ ]:
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF','TOTAL_VICTIMAS_24H',
       'TOTAL_VICTIMAS_30DF','TOTAL_HL24H','TOTAL_HL30DF']].loc[(df['TOTAL_MU24H'] == df['TOTAL_MU30DF']) & (df['TOTAL_HG24H']==df['TOTAL_MU30DF']) & (df['TOTAL_HG24H']==1)]

,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF,TOTAL_VICTIMAS_24H,TOTAL_VICTIMAS_30DF,TOTAL_HL24H,TOTAL_HL30DF
975,1,1,1,1,2,2,0,0
1303,1,1,1,1,2,2,0,0
1324,1,1,1,1,2,2,0,0
1332,1,1,1,1,2,2,0,0
1379,1,1,1,1,2,2,0,0
...,...,...,...,...,...,...,...,...
93982,1,1,1,1,2,2,0,0
94088,1,1,1,1,4,4,2,2
94451,1,1,1,1,2,2,0,0
97596,1,1,1,1,2,2,0,0


In [ ]:
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF','TOTAL_VICTIMAS_24H',
       'TOTAL_VICTIMAS_30DF','TOTAL_HL24H','TOTAL_HL30DF']].loc[(df['TOTAL_MU24H'] == 0) & (df['TOTAL_MU30DF']==df['TOTAL_HG24H']) & (df['TOTAL_HG30DF']==df['TOTAL_MU30DF']) & (df['TOTAL_MU30DF']==1)]

,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF,TOTAL_VICTIMAS_24H,TOTAL_VICTIMAS_30DF,TOTAL_HL24H,TOTAL_HL30DF


In [ ]:
df[['TOTAL_MU30DF','TOTAL_HG30DF','TOTAL_HL30DF','TOTAL_VICTIMAS_30DF']].loc[(df['TOTAL_MU30DF'] == df['TOTAL_HG30DF']) & (df['TOTAL_MU30DF'] > 1)]

,TOTAL_MU30DF,TOTAL_HG30DF,TOTAL_HL30DF,TOTAL_VICTIMAS_30DF
12034,2,2,2,6
12851,2,2,0,4
29576,2,2,6,10
30031,2,2,2,6
31432,3,3,0,6
83079,2,2,0,4
88434,2,2,6,10
91138,2,2,0,4


Es curioso que si que haya algunos casos donde hay más hospitalizados en 24horas que en 30 días, por lo que parece que en este caso estár hospitalizado en las primeras 24 horas no implica contar como que estuviste hospitalizado en los 30 días posteriores, no obstante son solo 232 filas, si los datos los estaban recogiendo de esa manera

In [ ]:
#mayor número de víctimas en 24 horas que en 30 días
df.loc[df['TOTAL_VICTIMAS_24H'] > df['TOTAL_VICTIMAS_30DF']]

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,...,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA


In [ ]:
#df = df.drop(['TOTAL_MU24H','TOTAL_VICTIMAS_24H'], axis=1)

Ya que en el dataset no se indican los tipos de lesion, diferenciaremos entre si un accidente fue leve o grave en base a si la víctima acudió al hospital o no lo hizo en 24 horas.

*Consultar Alex*

In [ ]:
df.columns

Index(['ID_ACCIDENTE', 'ANYO', 'MES', 'DIA_SEMANA', 'HORA', 'COD_PROVINCIA',
       'COD_MUNICIPIO', 'ISLA', 'ZONA', 'ZONA_AGRUPADA', 'CARRETERA', 'KM',
       'SENTIDO_1F', 'TITULARIDAD_VIA', 'TIPO_VIA', 'TIPO_ACCIDENTE',
       'TOTAL_MU24H', 'TOTAL_HG24H', 'TOTAL_HL24H', 'TOTAL_VICTIMAS_24H',
       'TOTAL_MU30DF', 'TOTAL_HG30DF', 'TOTAL_HL30DF', 'TOTAL_VICTIMAS_30DF',
       'TOTAL_VEHICULOS', 'TOT_PEAT_MU24H', 'TOT_BICI_MU24H',
       'TOT_CICLO_MU24H', 'TOT_MOTO_MU24H', 'TOT_TUR_MU24H', 'TOT_FURG_MU24H',
       'TOT_CAM_MENOS3500_MU24H', 'TOT_CAM_MAS3500_MU24H', 'TOT_BUS_MU24H',
       'TOT_OTRO_MU24H', 'TOT_SINESPECIF_MU24H', 'TOT_PEAT_MU30DF',
       'TOT_BICI_MU30DF', 'TOT_CICLO_MU30DF', 'TOT_MOTO_MU30DF',
       'TOT_TUR_MU30DF', 'TOT_FURG_MU30DF', 'TOT_CAM_MENOS3500_MU30DF',
       'TOT_CAM_MAS3500_MU30DF', 'TOT_BUS_MU30DF', 'TOT_VMP_MU30DF',
       'TOT_OTRO_MU30DF', 'TOT_SINESPECIF_MU30DF', 'NUDO', 'NUDO_INFO',
       'CARRETERA_CRUCE', 'PRIORI_NORMA', 'PRIORI_AGENTE', 'PR

# Ideas: 
- Agrupar columnas con fecha en una sola.
- Cambiar valores numéricos por el nomrbre de la variable, por ejemplo poner las comunidades en lugar de números

- Mortal = Fallece 
- Grave = Hospitalizado 30 días solo sí también está en la lista de 24 horas
- Leve = 24 horas hospitalizado (restar 30d - 24h)
- Sin lesiones = No fue al hospital